In [1]:
from sam import chdir_to_repopath
chdir_to_repopath()

from sam.stress_addition_model import (
    sam_prediction,
    get_sam_lcs,
    OLD_STANDARD
)
import os
from sam.dose_reponse_fit import dose_response_fit, ModelPredictions, FitSettings
from sam.plotting import plot_fit_prediction

from sam.plotting import plot_sam_prediction
from sam.data_formats import load_datapoints, read_data, load_files
from sam.system_stress import calc_system_stress
from sam.helpers import detect_hormesis_index
import matplotlib.pyplot as plt
import pandas as pd
from sam.plotting import *
import argparse
from tqdm import tqdm

PLOT_PATH = "control_imgs/sam_prediction"
SETTINGS = OLD_STANDARD

In [2]:
def sys_plot(model, example, setting, hormesis_index):
    cleaned_surv_func, system_stress_func = calc_system_stress(
        example.main_series, hormesis_index=hormesis_index, cfg=setting
    )

    cleaned_surv = cleaned_surv_func(model.concentration_curve) 
    cleaned_stress = setting.survival_to_stress(cleaned_surv)
    cleaned_surv *= example.meta.max_survival

    system_stress = system_stress_func(model.concentration_curve)
    system_surv = setting.stress_to_survival(system_stress) * example.meta.max_survival


    # Create the plot
    title = f"{example.meta.chemical} - {example.meta.organism}"

    fig, axs = plt.subplots(2, 2, figsize=(10, 6))


    def plot_surv(y, label, orig_series=None):
        plot_survival(
            model.concentration_curve,
            y,
            label = label,
            ax=axs[0, 0],
            orig_series=orig_series,
            xscale="linear",
            show_legend=False,
            xlab=None,
            ylab="Survival",
            title="Survival",
            color = None,
            hormesis_index=hormesis_index,
        )

        plot_survival(
            model.concentration_curve,
            y,
            label = label,
            ax=axs[1, 0],
            orig_series=orig_series,
            xscale="log",
            show_legend=False,
            xlab=None,
            ylab="Survival",
            color = None,
            hormesis_index=hormesis_index,
            title="Survival",
        )


    plot_surv(model.survival_curve, label="measurment", orig_series=model.inputs)
    plot_surv(cleaned_surv, label="without hormesis", orig_series=None)
    plot_surv(system_surv, label="system", orig_series=None)



    def plot_str(y, label):
        plot_stress(
            model.concentration_curve,
            y,
            label=label,
            ax=axs[0, 1],
            xscale="linear",
            show_legend=False,
            xlab=None,
            ylab="Stress",
            title="Stress",
            color =None,
        )

        plot_stress(
            model.concentration_curve,
            y,
            label=label,
            ax=axs[1, 1],
            xscale="log",
            show_legend=False,
            xlab=None,
            ylab="Stress",
            title="Stress",
            color =None,
        )

    plot_str(model.stress_curve, label="measurment")
    plot_str(cleaned_stress, label="without hormesis")
    plot_str(system_stress, label="system")

    axs[-1,-1].legend()

    if title:
        fig.suptitle(title)

    plt.tight_layout()
    return fig

In [3]:
save_dir = "control_imgs/hormesis_and_sys"

for path, data in tqdm(load_files()):
    setting = FitSettings(survival_max=data.meta.max_survival)

    model: ModelPredictions = dose_response_fit(
        data.main_series,
    )

    hormesis_index = detect_hormesis_index(data.main_series.survival_rate, tolerance=0.02)

        
    if hormesis_index is not None:
        fig = sys_plot(model, data, setting, hormesis_index)    

    else:
        title = f"{data.meta.chemical} - {data.meta.organism}"
        fig = plot_fit_prediction(model=model, title=title)

    # Save the plot
    save_path = os.path.join(save_dir, f"{os.path.split(path.replace('xlsx', 'png'))[1]}")
    fig.savefig(save_path)
    plt.close(fig)


 11%|█         | 2/18 [00:02<00:20,  1.27s/it]

Weibull 2-param fit failed, defaulting to linear regression
Weibull 3-param fit failed, defaulting to linear regression


 17%|█▋        | 3/18 [00:04<00:21,  1.45s/it]

Weibull 3-param fit failed, defaulting to linear regression


 28%|██▊       | 5/18 [00:07<00:17,  1.38s/it]

...

 61%|██████    | 11/18 [00:14<00:09,  1.35s/it]

Weibull 3-param fit failed, defaulting to linear regression


 94%|█████████▍| 17/18 [00:21<00:01,  1.20s/it]

Weibull 3-param fit failed, defaulting to linear regression


100%|██████████| 18/18 [00:23<00:00,  1.29s/it]
